In [30]:
import json
import pickle
import logging
import requests
import warnings
from app import app
from keras import models
from tensorflow import keras
warnings.filterwarnings("ignore")
from flask.testing import FlaskClient
from flask import Flask, request, jsonify
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [31]:
with open("gradient_boost_model.pkl", "rb") as file:
    gradient_boost_model = pickle.load(file)
gradient_boost_model

GradientBoostingClassifier()

In [33]:
with open("random_forest_model.pkl", "rb") as file:
    random_forest_model = pickle.load(file)
random_forest_model

RandomForestClassifier()

In [34]:
nn_model = keras.models.load_model("keras_model.h5")
nn_model

The '/' route returns a welcome message when you access the root URL.
The '/predict' route is designed to receive a POST request with a JSON payload containing features for prediction. It then uses the loaded machine learning models to make predictions based on the provided features.

Logging is crucial for monitoring and troubleshooting in a web application. It provides a record of events, such as received features and predictions, aiding in understanding the application's behavior and diagnosing issues effectively.

In [26]:
app = Flask("app")

# Configure logging
logging.basicConfig(filename='app.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

@app.route('/', methods=['GET'])
def index():
    return 'Hello, welcome to Hotel booking platform'

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get the request data from the user in JSON format
        request_json = request.get_json()

        #Define the input features
        features = {
            "lead_time": 30,
            "stays_in_week_nights": 3,
            "distribution_channel": "Online"
            }

        # Extract features from the request data
        features = request_json["features"]
        # Log the received features
        logging.info(f"Received features: {features}")

        # Make predictions using all three models
        features_array = np.array(features).reshape(1, -1)
        rf_prediction = random_forest_model.predict(features_array)[0]
        xgb_prediction = gradient_boost_model.predict(features_array)[0]
        nn_prediction = nn_model.predict(features_array)[0][0]

        # Log the prediction
        logging.info(f"Prediction: {prediction}")

        # Return the results as a JSON response
        return jsonify({
            "random_forest_prediction": int(rf_prediction),
            "xgboost_prediction": int(xgb_prediction),
            "neural_network_prediction": float(nn_prediction)
        })

    except Exception as e:
        # Log the error
        logging.error(f"Error: {str(e)}")
        return jsonify({'error': str(e)})

if "app" == '_main_':
    app.run(port=5000, debug=True)

In [27]:
@app.route('/get_url', methods=['GET'])
def get_url():
    # Get the base URL of the application
    base_url = request.base_url

    return jsonify({"url": base_url})  

In [28]:
get_url

<function __main__.get_url()>

In [35]:
features = {
    "lead_time": 30,
    "stays_in_week_nights": 3,
    "distribution_channel": "Online"
}

# Make a POST request to predict using the Flask app
response = requests.post("http://127.0.0.1:5000/predict_random_forest", json={"features": features})

# Print the prediction results
print(response.json())

features = {
    "lead_time": 30,
    "stays_in_week_nights": 3,
    "distribution_channel": "Online"
}

# Make a POST request to predict using the Flask app
response = requests.post("http://127.0.0.1:5000/predict_gradient_boost", json={"features": features})

# Print the prediction results
print(response.json())

features = {
    "lead_time": 30,
    "stays_in_week_nights": 3,
    "distribution_channel": "Online"
}

# Make a POST request to predict using the Flask app
response = requests.post("http://127.0.0.1:5000/predict_neural_networks", json={"features": features})

# Print the prediction results
print(response.json())

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /predict_random_forest (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f36f2466fe0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [21]:
client = FlaskClient(app)
features = {
            "lead_time": 30,
            "stays_in_week_nights": 3,
            "distribution_channel": "Online"
            }

# Now, use the client to make requests:
response = client.post("/predict", json={"features": features})


In [22]:
print(nn_prediction)

NameError: name 'nn_prediction' is not defined

In [ ]:
%run app.py